In [ ]:
# !pip install tensorflow-gpu==2.0.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import glob
import os

In [ ]:
# os.listdir('../input/cat-and-dog/training_set/training_set/')

In [ ]:
print('Tensorflow version: {}'.format(tf.__version__))

In [ ]:
# 本地图片的位置
# '../input/dc_2000/train/*/*.jpg'

train_image_path = glob.glob('../input/cat-and-dog/training_set/training_set/*/*.jpg')

len(train_image_path)# kaggle上8005张图片
train_image_path[-5:]

In [ ]:
np.random.shuffle(train_image_path) # 打乱路径, 前面几千张全是狗， 后面的全是猫

In [ ]:
# 本地数据集读取标签
# train_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[1].split('.')[0] == 'cat') for path in train_image_path]

# kaggle上的数据集读取标签
# ../input/cat-and-dog/training_set/training_set/cats/cat.2853.jpg
train_image_label = [int(path.split('/training_set/training_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in train_image_path]
train_image_label[-5:]

In [ ]:
# 查看读取到的图片是否正确
# image = tf.io.read_file(train_image_path[0])
# image = tf.image.decode_jpeg(image, channels=3)
# print(image.shape)
# plt.imshow(image)

In [ ]:
def load_preprocess_image(path, label):
    '''
    func:根据路径载入图像和label
    '''


    image = tf.io.read_file(path)   # 根据文件路径读取文件
    image = tf.image.decode_jpeg(image,channels=3) # jpg解码
    image = tf.image.resize(image, [256, 256]) # 统一大小
    
#    image = tf.image.random_crop(image, [256, 256, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.5)
    image = tf.image.random_contrast(image, 0, 1)
    image = tf.cast(image, tf.float32)  # 转换数据类型
    image = image/255   # 统一数据范围

    label = tf.reshape(label, (1,)) # reshape成(num,)
    return image, label

In [ ]:
#tf.image.convert_image_dtype

In [ ]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path, train_image_label)) # load数据集
# train_image_ds

In [ ]:
# 根据计算机的特征，自动的使用并行运算
AUTOTUNE = tf.data.experimental.AUTOTUNE
# AUTOTUNE

In [ ]:
# 预处理
train_image_ds = train_image_ds.map(load_preprocess_image, num_parallel_calls=AUTOTUNE) # 载入图像
# print(train_image_ds)

In [ ]:
# for img, label in train_image_ds.take(1):
#     plt.imshow(img)

In [ ]:
# 设定batch
BATCH_SIZE = 32
train_count = len(train_image_path) # 训练数据集的个数

In [ ]:
# 设立batch, train_image_ds是一个可迭代对象
train_image_ds = train_image_ds.shuffle(500).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)

In [ ]:
# imgs,labels=next(iter(train_image_ds))
# imgs.shape
# imgs[0]

In [ ]:
# train_image_path = glob.glob('../input/dc_2000/train/*/*.jpg')
# train_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[1].split('.')[0] == 'cat') for path in train_image_path]
# test_image_path = glob.glob('../input/dc_2000/test/*/*.jpg')
# test_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[0].split('.')[0] == 'cat') for path in test_image_path]

# train_image_path = glob.glob('../input/cat-and-dog/training_set/training_set/*/*.jpg')
test_image_path = glob.glob('../input/cat-and-dog/test_set/test_set/*/*.jpg')
# train_image_label = [int(path.split('/training_set/training_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in train_image_path]
test_image_label = [int(path.split('/test_set/test_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in test_image_path]

# 载入测试图片
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path, test_image_label))
test_image_ds = test_image_ds.map(load_preprocess_image, num_parallel_calls=AUTOTUNE)
test_image_ds = test_image_ds.batch(BATCH_SIZE)
test_image_ds = test_image_ds.prefetch(AUTOTUNE)

In [ ]:
len(test_image_path)

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

odel: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param \#   
=================================================================
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 512)       1180160   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 512)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 1024)      4719616   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
=================================================================
Total params: 6,533,249
Trainable params: 6,533,249
Non-trainable params: 0
_________________________________________________________________

In [ ]:
# model.summary()

In [ ]:
# tf.keras.losses.binary_crossentropy([0.,0.,1.,1.], [1.,1.,1.,1.])#二元交叉熵定义loss

In [ ]:
# [https://www.jianshu.com/p/aebcaf8af76e]
# 使用Adam优化, 具体一些的对Adam的解释可以看上面的博客
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
epoch_loss_avg = tf.keras.metrics.Mean('train_loss') # 
train_accuracy = tf.keras.metrics.Accuracy() # 计算正确率

epoch_loss_avg_test = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.Accuracy()

In [ ]:
# train_accuracy([1,0,1], [1,1,1])

In [ ]:
def train_step(model, images, labels):
    """
    func:训练集求梯度
    """
    with tf.GradientTape() as t:
        pred = model(images) # 使用模型预测结果
        loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred) # 计算损失值
    
    grads = t.gradient(loss_step, model.trainable_variables) #计算dloss/d(参数)
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #根据dloss/d(variable)进行优化

    epoch_loss_avg(loss_step)# 计算loss的均值
    train_accuracy(labels, tf.cast(pred>0, tf.int32))# 计算精确度

In [ ]:
def test_step(model, images, labels):
    pred = model(images, training=False)
    loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred)
    epoch_loss_avg_test(loss_step)
    test_accuracy(labels, tf.cast(pred>0, tf.int32))

In [ ]:
train_loss_results = []
train_acc_results = []

test_loss_results = []
test_acc_results = []

In [ ]:
num_epochs = 30

In [ ]:
for epoch in range(num_epochs):
#     print(train_image_ds)
    for imgs_, labels_ in train_image_ds:
        train_step(model, imgs_, labels_)
        print('.', end='')
    print()
    
    train_loss_results.append(epoch_loss_avg.result())
    train_acc_results.append(train_accuracy.result())
    
    
    for imgs_, labels_ in test_image_ds:
        test_step(model, imgs_, labels_)
        
    test_loss_results.append(epoch_loss_avg_test.result())
    test_acc_results.append(test_accuracy.result())
    
    print('Epoch:{}: loss: {:.3f}, accuracy: {:.3f}, test_loss: {:.3f}, test_accuracy: {:.3f}'.format(
        epoch + 1,
        epoch_loss_avg.result(),
        train_accuracy.result(),
        epoch_loss_avg_test.result(),
        test_accuracy.result()
    ))
    
    epoch_loss_avg.reset_states()
    train_accuracy.reset_states()
    
    epoch_loss_avg_test.reset_states()
    test_accuracy.reset_states()

...........................................................................................................................................................................................................................................................
Epoch:1: loss: 0.690, accuracy: 0.516, test_loss: 0.684, test_accuracy: 0.545
...........................................................................................................................................................................................................................................................
Epoch:2: loss: 0.683, accuracy: 0.540, test_loss: 0.679, test_accuracy: 0.551
...........................................................................................................................................................................................................................................................
Epoch:3: loss: 0.681, accuracy: 0.547, test_loss: 0.678, test_accuracy: 0.571
...........................................................................................................................................................................................................................................................
Epoch:4: loss: 0.672, accuracy: 0.573, test_loss: 0.674, test_accuracy: 0.558
...........................................................................................................................................................................................................................................................
Epoch:5: loss: 0.669, accuracy: 0.572, test_loss: 0.669, test_accuracy: 0.559
...........................................................................................................................................................................................................................................................
Epoch:6: loss: 0.661, accuracy: 0.589, test_loss: 0.656, test_accuracy: 0.597
...........................................................................................................................................................................................................................................................
Epoch:7: loss: 0.657, accuracy: 0.595, test_loss: 0.647, test_accuracy: 0.609
...........................................................................................................................................................................................................................................................
Epoch:8: loss: 0.647, accuracy: 0.612, test_loss: 0.650, test_accuracy: 0.607
...........................................................................................................................................................................................................................................................
Epoch:9: loss: 0.637, accuracy: 0.625, test_loss: 0.639, test_accuracy: 0.634
...........................................................................................................................................................................................................................................................
Epoch:10: loss: 0.634, accuracy: 0.619, test_loss: 0.632, test_accuracy: 0.627
...........................................................................................................................................................................................................................................................
Epoch:11: loss: 0.620, accuracy: 0.644, test_loss: 0.613, test_accuracy: 0.662
...........................................................................................................................................................................................................................................................
Epoch:12: loss: 0.608, accuracy: 0.656, test_loss: 0.616, test_accuracy: 0.645
...........................................................................................................................................................................................................................................................
Epoch:13: loss: 0.607, accuracy: 0.655, test_loss: 0.599, test_accuracy: 0.666
...........................................................................................................................................................................................................................................................
Epoch:14: loss: 0.599, accuracy: 0.666, test_loss: 0.592, test_accuracy: 0.674
...........................................................................................................................................................................................................................................................
Epoch:15: loss: 0.595, accuracy: 0.672, test_loss: 0.602, test_accuracy: 0.663
...........................................................................................................................................................................................................................................................
Epoch:16: loss: 0.579, accuracy: 0.688, test_loss: 0.578, test_accuracy: 0.683
...........................................................................................................................................................................................................................................................
Epoch:17: loss: 0.582, accuracy: 0.687, test_loss: 0.567, test_accuracy: 0.699
...........................................................................................................................................................................................................................................................
Epoch:18: loss: 0.581, accuracy: 0.679, test_loss: 0.575, test_accuracy: 0.701
...........................................................................................................................................................................................................................................................
Epoch:19: loss: 0.572, accuracy: 0.692, test_loss: 0.572, test_accuracy: 0.694
...........................................................................................................................................................................................................................................................
Epoch:20: loss: 0.559, accuracy: 0.704, test_loss: 0.559, test_accuracy: 0.705
...........................................................................................................................................................................................................................................................
Epoch:21: loss: 0.547, accuracy: 0.711, test_loss: 0.561, test_accuracy: 0.703
...........................................................................................................................................................................................................................................................
Epoch:22: loss: 0.558, accuracy: 0.707, test_loss: 0.568, test_accuracy: 0.692
...........................................................................................................................................................................................................................................................
Epoch:23: loss: 0.534, accuracy: 0.724, test_loss: 0.550, test_accuracy: 0.726
...........................................................................................................................................................................................................................................................
Epoch:24: loss: 0.536, accuracy: 0.719, test_loss: 0.536, test_accuracy: 0.728
...........................................................................................................................................................................................................................................................
Epoch:25: loss: 0.523, accuracy: 0.729, test_loss: 0.544, test_accuracy: 0.724
...........................................................................................................................................................................................................................................................
Epoch:26: loss: 0.516, accuracy: 0.736, test_loss: 0.536, test_accuracy: 0.716
...........................................................................................................................................................................................................................................................
Epoch:27: loss: 0.512, accuracy: 0.741, test_loss: 0.545, test_accuracy: 0.719
...........................................................................................................................................................................................................................................................
Epoch:28: loss: 0.500, accuracy: 0.740, test_loss: 0.492, test_accuracy: 0.759
...........................................................................................................................................................................................................................................................
Epoch:29: loss: 0.483, accuracy: 0.759, test_loss: 0.505, test_accuracy: 0.738
...........................................................................................................................................................................................................................................................
Epoch:30: loss: 0.481, accuracy: 0.755, test_loss: 0.487, test_accuracy: 0.750

In [ ]:
print(len(train_acc_results))
print(len(test_acc_results))
plt.plot(range(1,num_epochs+1),train_acc_results,label='train_accuracy')
plt.plot(range(1,num_epochs+1),test_acc_results,label='test_accuracy')
plt.legend()

In [ ]:
plt.plot(range(1,num_epochs+1),train_loss_results,label='train_loss')
plt.plot(range(1,num_epochs+1),test_loss_results,label='test_loss')
plt.legend()